## Load and Explore  Labels Data

In [2]:
#Import labeled data downloaded from : http://gapfinder-tools.wmflabs.org/admin/sectionalignment/userinput/
#A copy of can be downlodaded from: ADD GITHUB LINK HERE
import pandas as pd

labels = pd.read_csv('SectionTranslationLabels.tsv',sep='\t')
labels['destination_title'] = labels.destination_title.map(eval) #transform string to list

In [3]:
total = 0
for src,data in labels.groupby(labels.source_lang):
    for dest,data2 in data.groupby('destination_language'):
        print(src,dest,len(data2))
        total+=len(data2)

ar en 544
ar es 87
ar fr 276
ar ja 11
ar ru 27
en ar 473
en es 999
en fr 999
en ja 698
en ru 947
es ar 3
es en 526
es fr 500
es ja 1
es ru 19
fr ar 42
fr en 529
fr es 142
fr ja 76
fr ru 52
ja en 100
ja es 10
ja fr 77
ru ar 3
ru en 418
ru es 18
ru fr 332
ru ja 37


In [28]:
labels

,source_lang,source_title,destination_language,destination_title,user_session_key
0,ru,Родились,ja,[出自],1hl2ay9d01lek5piry4jjqvj377xwmh7
1,ru,Этимология,ja,[由来],1hl2ay9d01lek5piry4jjqvj377xwmh7
2,ru,Комментарии,ja,[註釈],1hl2ay9d01lek5piry4jjqvj377xwmh7
3,ru,Культура,ja,[文化],1hl2ay9d01lek5piry4jjqvj377xwmh7
4,ru,Источник,ja,[出典],1hl2ay9d01lek5piry4jjqvj377xwmh7
5,ru,Турнирная таблица,ja,[順位表],1hl2ay9d01lek5piry4jjqvj377xwmh7
6,ru,Медалисты,ja,"[受賞者, メダリスト]",1hl2ay9d01lek5piry4jjqvj377xwmh7
7,ru,Города-побратимы,ja,[姉妹都市],1hl2ay9d01lek5piry4jjqvj377xwmh7
8,ru,Улицы,ja,"[道路, 通り]",1hl2ay9d01lek5piry4jjqvj377xwmh7
9,ru,Расположение,ja,"[位置, 場所]",1hl2ay9d01lek5piry4jjqvj377xwmh7


In [4]:
### Exploid multiple translatios
df = labels.set_index(['source_lang', 'source_title','destination_language'])['destination_title'].apply(pd.Series).stack()
df = df.reset_index()
del(df['level_3'])
df.rename(columns={0:'destination_title'},inplace=True)
df.set_index('source_lang',inplace=True)
labelsExploid = df.reset_index() 

In [5]:
total = 0
for src,data in df.groupby(df.index):
    for dest,data2 in data.groupby('destination_language'):
        print(src,dest,len(data2))
        total+=len(data2)
print(total)

ar en 565
ar es 87
ar fr 285
ar ja 21
ar ru 29
en ar 543
en es 1062
en fr 1210
en ja 1169
en ru 1178
es ar 3
es en 569
es fr 522
es ja 2
es ru 19
fr ar 43
fr en 746
fr es 155
fr ja 162
fr ru 64
ja en 129
ja es 12
ja fr 105
ru ar 3
ru en 460
ru es 19
ru fr 346
ru ja 41
9549


# From This point is test , clean later

## Classification 

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import editdistance
translations =pd.read_pickle('translations.p')

def getCosineSim(A,B): #Avoiding cosine_similarty problems with Nan and Series shape
    A = A.values.reshape(1,A.shape[0])
    B = B.values.reshape(1,B.shape[0])

    try:   return cosine_similarity(A,B)[0][0]
    except: return 0 
    
def balanceDF(df,column,ratio,valToKeep):
    '''
    Receive dataframe with a Binary column and return a subset 
    balanced in that column acording to ratio
    valToKeep value to keep
    '''
    minCol  = min(df[column].value_counts())
    rowsToKeep = df[df[column] == valToKeep]
    df = df.sample(minCol*ratio,random_state=5)
    return pd.concat([df,rowsToKeep])

def prepareDataNew(langFrom,langTo,AutomaticTranslation=True):
    global labelsExploid
    labelsPair = labelsExploid[(labelsExploid['source_lang'] == langFrom) & (labelsExploid['destination_language'] ==langTo )][['source_title','destination_title']]
    labeledSec = labelsPair.source_title.unique()
    try:
        distances = pd.read_pickle('FastTextDistances/distances_%s_%s.p' % (langFrom,langTo))
    except:
        distances = pd.read_pickle('FastTextDistances/distances_%s_%s.p' % (langTo,langFrom))
    try:
        mydistances = pd.read_pickle('FastTextDistances-my_alingments/distances_%s_%s.p' % (langFrom,langTo))
    except:
        mydistances = pd.read_pickle('FastTextDistances-my_alingments//distances_%s_%s.p' % (langTo,langFrom))
    try:
        linkdistances = pd.read_pickle('linksSimilarities/%s_%s.p' % (langFrom,langTo))
    except:
        linkdistances = pd.read_pickle('linksSimilarities/%s_%s.p' % (langTo,langFrom))
    try:
        contentdistances = pd.read_pickle('fastTextContentSimilarities/%s_%s.p' % (langTo,langFrom))
    except:
        contentdistances = pd.read_pickle('fastTextContentSimilarities/%s_%s.p' % (langFrom,langTo))

    ##langFrom is documents, LangTo is words,
    coOcurrences = pd.read_pickle('resultsMappingTfIdf/TfIDF_%s_%s.p' % (langFrom,langTo))
    #CLEANING NOISE, THIS SHOULD BE DONE BEFORE
    coOcurrences =  coOcurrences[~coOcurrences[langFrom].str.contains('\[|{|"')]  #Cleaning sections with links
    coOcurrences =  coOcurrences[~coOcurrences[langTo].str.contains('\[|{|"')]  #Cleaning sections with links

    features = pd.merge(distances, coOcurrences,  how='inner', 
                        left_on=[langFrom,langTo], right_on = [langFrom,langTo])
    features = features[features[langFrom].isin(labeledSec)]
    features = pd.merge(features, mydistances,  how='inner', 
                        left_on=[langFrom,langTo], right_on = [langFrom,langTo],suffixes=['','_my']) 
    features = pd.merge(features, contentdistances,  how='inner', 
                        left_on=[langFrom,langTo], right_on = [langFrom,langTo])
    features = pd.merge(features, linkdistances,  how='inner', 
                        left_on=[langFrom,langTo], right_on = [langFrom,langTo])
    features = features[features[langFrom].isin(labeledSec)]
    dataset = pd.merge(features, labelsPair[['source_title','destination_title']], how='left', left_on=[langFrom,langTo], right_on = ['source_title','destination_title'])#,'destination_title'])
                                
    dataset['isTranslation'] = dataset.destination_title.notna()
    dataset.rename(columns={langTo+'_tfidf':'LangTo_tfidf',langFrom+'_tfidf':'LangFrom_tfidf',langFrom:'langFrom',langTo:'langTo'},inplace=True)
    del(dataset['source_title'])
    del(dataset['destination_title'])
    secsFrom = pd.read_pickle('SectionsCharacterization/sections-articles_%s.p' % langFrom )
    secsTo = pd.read_pickle('SectionsCharacterization/sections-articles_%s.p' % langTo )
    secsFromJoined = pd.merge(dataset, secsFrom, how='left', left_on='langFrom', right_on = secsFrom.index)#,'destination_title'])
    secsToJoined    = pd.merge(dataset, secsTo, how='left', left_on='langTo', right_on = secsTo.index,suffixes=('', '_langTo'))#,'destination_title'])
    colsSec = secsFrom.columns
    secDistSub  = secsFromJoined[colsSec] - secsToJoined[colsSec]
    secDistMult  = secsFromJoined[colsSec] * secsToJoined[colsSec]
    #Removing Cosine Similarity, take long time and it is not adding enough information

    #secsJoined = secsFromJoined[colsSec].join(secsToJoined[colsSec],rsuffix='_r')
    #dataset['DistCos'] = secsJoined.apply(lambda row: getCosineSim(row[cols],row[[c+'_r' for c in cols]]),axis=1)                                              
    dataset = dataset.join(secDistSub).join(secDistMult,rsuffix='_mult')
    dataset = dataset.join(secsFrom[['rank']],on='langFrom',rsuffix='_from')

    dataset['editDistance'] = dataset.apply(lambda x:editdistance.eval(x['langFrom'],x['langTo']) / max(len(x['langFrom']),len(x['langTo'])) ,axis=1)
    if AutomaticTranslation:
        trans = translations[(translations['source_lang'] == langFrom) & (translations['destination_language'] == langTo)][['source_title','automatic_translation']]
        dataset = pd.merge(dataset, trans, how='left', left_on='langFrom', right_on = 'source_title')#,'destination_title'])
        dataset['isAutomaticTranslationFrom'] = dataset['langTo'] == dataset['automatic_translation']
        del(dataset['source_title'])
        del(dataset['automatic_translation'])
        trans = translations[(translations['source_lang'] == langTo) & (translations['destination_language'] == langFrom)][['source_title','automatic_translation']]
        dataset = pd.merge(dataset, trans, how='left', left_on='langTo', right_on = 'source_title')#,'destination_title'])
        dataset['isAutomaticTranslationTo'] = dataset['langFrom'] == dataset['automatic_translation']
        del(dataset['source_title'])
        del(dataset['automatic_translation'])
    #Scaling to one 
    dataset['count_normed']  = dataset['count'] / dataset['count'].max()
    dataset['LangFrom_tfidf_normed']  = dataset['LangFrom_tfidf'] / dataset['LangFrom_tfidf'].max()
    dataset['LangTo_tfidf_normed']  = dataset['LangTo_tfidf'] / dataset['LangTo_tfidf'].max()
    #ranks (relative position, instead of raw number, per word)
    dataset["rank_by_count"] = dataset.groupby("langFrom")["count"].rank("dense", ascending=False)
    dataset["rank_by_LangFrom_tfidf"] = dataset.groupby("langFrom")["LangFrom_tfidf"].rank("dense", ascending=False)
    dataset["rank_by_LangTo_tfidf"] = dataset.groupby("langFrom")["LangTo_tfidf"].rank("dense", ascending=False)

    #polinomial  / weighted 
    dataset['Distance_myWeighted']  = dataset['count_normed'] *  dataset['Distance_my']
    dataset['DistanceWeighted']  = dataset['count_normed'] *  dataset['Distance']
    dataset['LangFrom_tfidfWeighted']  = dataset['count_normed'] *  dataset['LangFrom_tfidf']
    dataset['LangTo_tfidfWeighted']  = dataset['count_normed'] *  dataset['LangTo_tfidf']
    return dataset


def prepareDataNewLangNoLabels(langFrom,langTo,AutomaticTranslation=False):
    secs = pd.read_pickle('newLangsSectionsToConsider/%s.p' % langFrom)
    try:
        distances = pd.read_pickle('FastTextDistances/distances_%s_%s.p' % (langFrom,langTo))
    except:
        distances = pd.read_pickle('FastTextDistances/distances_%s_%s.p' % (langTo,langFrom))
    try:
        mydistances = pd.read_pickle('FastTextDistances-my_alingments/distances_%s_%s.p' % (langFrom,langTo))
    except:
        mydistances = pd.read_pickle('FastTextDistances-my_alingments/distances_%s_%s.p' % (langTo,langFrom))
    try:
        linkdistances = pd.read_pickle('linksSimilarities/%s_%s.p' % (langFrom,langTo))
    except:
        linkdistances = pd.read_pickle('linksSimilarities/%s_%s.p' % (langTo,langFrom))
        
    ##langFrom is documents, LangTo is words,
    coOcurrences = pd.read_pickle('resultsMappingTfIdf/TfIDF_%s_%s.p' % (langFrom,langTo))
    coOcurrences = pd.merge(secs, coOcurrences,  how='inner', 
                        left_on=langFrom, right_on = langFrom)
    #CLEANING NOISE, THIS SHOULD BE DONE BEFORE
    coOcurrences =  coOcurrences[~coOcurrences[langFrom].str.contains('\[')]  #Cleaning sections with links
    coOcurrences =  coOcurrences[~coOcurrences[langTo].str.contains('\[')]  #Cleaning sections with links

    features = pd.merge(distances, coOcurrences,  how='inner', 
                        left_on=[langFrom,langTo], right_on = [langFrom,langTo])
    features = pd.merge(features, mydistances,  how='inner', 
                        left_on=[langFrom,langTo], right_on = [langFrom,langTo],suffixes=['','_my']) 
    features = pd.merge(features, linkdistances,  how='inner', 
                        left_on=[langFrom,langTo], right_on = [langFrom,langTo])
    dataset = features
    dataset.rename(columns={langTo+'_tfidf':'LangTo_tfidf',langFrom+'_tfidf':'LangFrom_tfidf',langFrom:'langFrom',langTo:'langTo'},inplace=True)
    secsFrom = pd.read_pickle('SectionsCharacterization/sections-articles_%s.p' % langFrom )
    del(secsFrom['TotalCount']) #Need to rerun section chacterization for the old training langs
    secsTo = pd.read_pickle('SectionsCharacterization/sections-articles_%s.p' % langTo )
    secsFromJoined = pd.merge(dataset, secsFrom, how='left', left_on='langFrom', right_on = secsFrom.index)#,'destination_title'])
    secsToJoined    = pd.merge(dataset, secsTo, how='left', left_on='langTo', right_on = secsTo.index,suffixes=('', '_langTo'))#,'destination_title'])
    colsSec = secsFrom.columns
    secDistSub  = secsFromJoined[colsSec] - secsToJoined[colsSec]
    secDistMult  = secsFromJoined[colsSec] * secsToJoined[colsSec]
    #Removing Cosine Similarity, take long time and it is not adding enough information

    #secsJoined = secsFromJoined[colsSec].join(secsToJoined[colsSec],rsuffix='_r')
    #dataset['DistCos'] = secsJoined.apply(lambda row: getCosineSim(row[cols],row[[c+'_r' for c in cols]]),axis=1)                                              
    dataset = dataset.join(secDistSub).join(secDistMult,rsuffix='_mult')
    dataset['editDistance'] = dataset.apply(lambda x:editdistance.eval(x['langFrom'],x['langTo']) / max(len(x['langFrom']),len(x['langTo'])) ,axis=1)
    if AutomaticTranslation:
        trans = translations[(translations['source_lang'] == langFrom) & (translations['destination_language'] == langTo)][['source_title','automatic_translation']]
        dataset = pd.merge(dataset, trans, how='left', left_on='langFrom', right_on = 'source_title')#,'destination_title'])
        dataset['isAutomaticTranslationFrom'] = dataset['langTo'] == dataset['automatic_translation']
        del(dataset['source_title'])
        del(dataset['automatic_translation'])
        trans = translations[(translations['source_lang'] == langTo) & (translations['destination_language'] == langFrom)][['source_title','automatic_translation']]
        dataset = pd.merge(dataset, trans, how='left', left_on='langTo', right_on = 'source_title')#,'destination_title'])
        dataset['isAutomaticTranslationTo'] = dataset['langFrom'] == dataset['automatic_translation']
        del(dataset['automatic_translation'])
    #Polynomial  features, weighted by coocurences
    dataset['count_normed']  = dataset['count'] / dataset['count'].max()
    dataset['Distance_myWeighted']  = dataset['count_normed'] *  dataset['Distance_my']
    dataset['DistanceWeighted']  = dataset['count_normed'] *  dataset['Distance']
    dataset['LangFrom_tfidfWeighted']  = dataset['count_normed'] *  dataset['LangFrom_tfidf']
    dataset['LangTo_tfidfWeighted']  = dataset['count_normed'] *  dataset['LangTo_tfidf']
    return dataset

/usr/lib/python3/dist-packages/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 30] Read-only file system.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [7]:
from itertools import permutations
langs=['ru', 'ja', 'fr', 'ar','es', 'en']
features = ['LangTo_tfidf','LangFrom_tfidf','Distance','Distance_my',
            'count','pos_norm_avg','pos_inv_avg','pos_norm_median',
            'pos_avg_mult','size_median_mult','num_links_avg',
           'links_jaccardMEAN', 'links_jaccardMEDIAN',
           'links_jaccardSUM', 'links_commonMEAN', 'links_commonMEDIAN',
           'links_commonSUM','editDistance','count_normed',
            'Distance_myWeighted','DistanceWeighted','LangFrom_tfidfWeighted','LangTo_tfidfWeighted']
            #'isAutomaticTranslationTo','isAutomaticTranslationFrom']
finalResults = []
datasets = {}
for pair in permutations(langs,2):
        print(pair)
        try: #for avoiding error in empty pairs (ja,ru)
            tmpDF = prepareDataNew(pair[0],pair[1])
            if tmpDF.shape[0] > 1: #to deal with empty pairs
                datasets[pair[0]] = datasets.get(pair[0],{})
                datasets[pair[0]][pair[1]] = balanceDF(tmpDF,'isTranslation',2,True)
        except:
            print('error')

('ru', 'ja')
('ru', 'fr')
('ru', 'ar')
('ru', 'es')
('ru', 'en')
('ja', 'ru')
error
('ja', 'fr')
('ja', 'ar')
error
('ja', 'es')
('ja', 'en')
('fr', 'ru')
('fr', 'ja')
('fr', 'ar')
('fr', 'es')
('fr', 'en')
('ar', 'ru')
('ar', 'ja')
('ar', 'fr')
('ar', 'es')
('ar', 'en')
('es', 'ru')
('es', 'ja')
('es', 'fr')
('es', 'ar')
('es', 'en')
('en', 'ru')
('en', 'ja')
('en', 'fr')
('en', 'ar')
('en', 'es')


In [64]:
#Create prediction
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from itertools import permutations

for lang in ['fa']:#langs:
    print(lang)
    tmpDF = []
    for lang1,data in datasets.items():
        if lang1 != lang:
            for lang2,dataset in data.items():
                if lang2 != lang:
                    tmpDF.append(dataset)    
    TrainData = pd.concat(tmpDF)
    X_train = TrainData[features]
    Y_train = TrainData['isTranslation']
    Model = XGBClassifier()
    Model.fit(X_train, Y_train)
    print(sorted(zip(Model.feature_importances_,X_train.columns),reverse=True))
    for lang1,lang2 in [('fa','en')]: #permutations(langs,2):
        if lang1 == lang:
                TestData = prepareDataNewLangNoLabels(lang1,lang2) # I need to create again the dataset, because the stored ones, are balanaced
                TestData  = TestData[TestData['count'] > 5] # PRE-FILTER, CANDIDATES NEWS TO COOCURR AT LEAST FIVE (Arbitrary), THIS FOR REMOVING TYPOS AND NOISE
                X_test = TestData[features]
                y_prob= Model.predict_proba(X_test)
                TestData['prob'] = [pred[1] for pred in y_prob]
                output = []
                for secFrom,data in TestData.groupby('langFrom'):
                        #### This is for create spreadshets with examples
                        data = data.sort_values('prob',ascending=False)[['langTo','prob','count']]
                        data.reset_index(inplace=True)
                        data['rank'] = data.index +1 
                        candidates = data.iloc[0:5]
                        candidates['secFrom'] =secFrom
                        candidates.set_index('secFrom',inplace=True)
                        candidates.drop_duplicates(inplace=True)
                        candidates.sort_values('prob',ascending=False,inplace=True)
                        del(candidates['index'])
                        output.append(candidates)
                        
                            
                spreadsheetOutput = pd.concat(output)
                spreadsheetOutput.rename(columns={'isTranslation':'label'},inplace=True)
                spreadsheetOutput.to_excel('prectionsNoLabels/%s_to_%s.xls' % (lang1,lang2))



fa


/home/dsaez/venv/lib/python3.5/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


[(0.15582451, 'Distance_my'), (0.11043873, 'LangFrom_tfidf'), (0.090771556, 'LangTo_tfidfWeighted'), (0.0892587, 'links_jaccardMEAN'), (0.0816944, 'Distance'), (0.07261725, 'size_median_mult'), (0.07261725, 'LangTo_tfidf'), (0.06354009, 'editDistance'), (0.051437218, 'links_jaccardSUM'), (0.04236006, 'pos_inv_avg'), (0.033282906, 'pos_norm_avg'), (0.030257186, 'num_links_avg'), (0.018154312, 'pos_norm_median'), (0.016641453, 'links_jaccardMEDIAN'), (0.015128593, 'Distance_myWeighted'), (0.010590015, 'pos_avg_mult'), (0.010590015, 'links_commonMEAN'), (0.009077156, 'LangFrom_tfidfWeighted'), (0.0075642965, 'count'), (0.0075642965, 'DistanceWeighted'), (0.004538578, 'links_commonMEDIAN'), (0.004538578, 'count_normed'), (0.0015128592, 'links_commonSUM')]


/home/dsaez/venv/lib/python3.5/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dsaez/venv/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [44]:
spreadsheetOutput

,langTo,prob,count,rank
secFrom,,,,
آب و هوا,Clima,0.704273,43,1
آب و هوا,Recursos naturales,0.253927,6,2
آب و هوا,Lugares de interés,0.244641,6,3
آب و هوا,Transporte,0.144905,30,4
آب و هوا,Infraestructura,0.120630,9,5
آثار,Obras,0.983210,256,1
آثار,Libros,0.962704,20,2
آثار,Filmografía,0.950563,53,3
آثار,Discografía,0.942169,42,4


In [55]:
#No Labels translator
#CHECK THIS 
from time import sleep 
from googletrans import Translator
import random
import requests
import pandas as pd
#### Proxies for use googletrans #####


NoLabelstranslations = pd.DataFrame([]) # Init translations results
secs = pd.read_pickle('newLangsSectionsToConsider/%s.p' % 'fa')


translator = Translator(service_urls=['translate.google.co.uk'], timeout=15)
result = translator.translate(secs['fa'].tolist(),src='fa',dest='en')

pairs = [(lang,sec.origin,lang2,sec.text) for sec in result]
pairs = pd.DataFrame(pairs,columns=['source_lang','source_title','destination_language','automatic_translation'])


In [66]:
spreadsheetOutput.join(pairs[['source_title','automatic_translation']].set_index('source_title')).to_excel('prectionsNoLabels/fa_to_en_NOGoogleTranslate.xls')

In [59]:
pairs

,source_lang,source_title,destination_language,automatic_translation
0,fa,منابع,en,References
1,fa,جستارهای وابسته,en,depended queries
2,fa,پیوند به بیرون,en,Link to the outside
3,fa,جمعیت,en,the population
4,fa,پانویس,en,Footnote
5,fa,خصوصیات,en,Specification
6,fa,نگارخانه,en,Gallery
7,fa,بازیگران,en,Actors
8,fa,تاریخچه,en,History
9,fa,زندگی‌نامه,en,"biography, diary"
